In [25]:
from dotenv import load_dotenv
load_dotenv()

True

In [26]:
# Path to your image
image_path = "data/224 Rue Robert Saint-Bruno-de-Montarville.png"
excel_path = image_path.replace(".png", ".xlsx")

In [27]:
import base64

def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

# Getting the base64 string
base64_image = encode_image(image_path)

In [28]:
import requests
import os
from io import StringIO

api_key = os.environ.get("OPENAI_API_KEY")

headers = {
  "Content-Type": "application/json",
  "Authorization": f"Bearer {api_key}"
}

prompt = """
Extract the information in the table of this image.
Return the data in a format that can be ingested with Pandas. 
Just return the data, nothing else. 
Split column using ; as separator.

Code:
df = pd.read_csv(StringIO(data), header=0, names=['Pièce', 'Dimensions'])

your answer:
data =
"""

payload = {
  "model": "gpt-4-vision-preview",
  "messages": [
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": prompt
        },
        {
          "type": "image_url",
          "image_url": {
            "url": f"data:image/jpeg;base64,{base64_image}"
          }
        }
      ]
    }
  ],
  "max_tokens": 300
}

response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
data = response.json()['choices'][0]['message']['content']
print(data)

data =
Pièce;Dimensions
Hall d'entrée/Vestibule;4,8 X 4,8 p
Salon;14,2 X 13,7 p irr
Salle à manger;19,6 X 13,1 p irr
Cuisine;17,4 X 11 p
Coin-repas;9,7 X 6,7 p
Salle familiale;24,10 X 15,6 p
Chambre à coucher;11,5 X 9,7 p
Salle d'eau;8,1 X 5,9 p
Chambre à coucher principale;15,6 X 12,6 p
Chambre à coucher;11,6 X 9,10 p
Chambre à coucher;10,2 X 9,8 p
Salle de bains;10,11 X 9,11 p
Salle de jeux;25,7 X 16,3 p irr


SyntaxError: unterminated string literal (detected at line 2) (2483477428.py, line 2)

In [30]:
import pandas as pd
df = pd.read_csv(StringIO(data.split('"""')[1]), sep=';', header=0, names=['Pièce', 'Dimensions'])
df

IndexError: list index out of range

In [ ]:
import pandas as pd
import re

def convert_comma_to_dot(text):
    # Replace comma with dot to convert to float
    return text.replace(',', '.')

def extract_numbers(text):
    # Improved regex to extract dimensions in the format 'number.number'
    numbers_dots_pattern = re.compile(r'[0-9.,]+')
    extracted = numbers_dots_pattern.findall(text)
    # Convert comma to dot
    extracted = [convert_comma_to_dot(number) for number in extracted]
    return extracted

def extract_dimensions(text):
    extracted = extract_numbers(text)
    if len(extracted) == 2:
        return extracted
    else:
        # Handle cases where the extraction does not yield exactly 2 values
        return [None, None]

# Assuming df is your DataFrame
# Apply extract_dimensions and create a new DataFrame from the results
dimensions_df = df['Dimensions'].apply(extract_dimensions).apply(pd.Series)

# Rename the new columns and add them to the original DataFrame
df['width'] = dimensions_df[0]
df['length'] = dimensions_df[1]

In [ ]:
df

,Pièce,Dimensions,width,length
0,Salon,"10,8 X 17,8 p",10.8,17.8
1,Salle à manger,"10,7 X 10,11 p",10.7,10.11
2,Cuisine,"10,4 X 14,2 p",10.4,14.2
3,Chambre à coucher principale,"10,9 X 16,11 p",10.9,16.11
4,Chambre à coucher,"12,9 X 10,7 p",12.9,10.7
5,Salle de bains,"5,4 X 8 p",5.4,8
6,Chambre à coucher,"7,11 X 10 p",7.11,10
7,Bureau,"8,9 X 7,1 p",8.9,7.1
8,Salle familiale,"13,3 X 17,11 p",13.3,17.11
9,Salle de lavage,"4,9 X 10,11 p",4.9,10.11


In [ ]:
def save_to_excel(df, excel_path=excel_path):
    df.to_excel(excel_path, index=False)

save_to_excel(df)